In [2]:
import pandas as pd
df = pd.read_csv('data/new_hack.csv')

In [3]:
!pip install -U sentence-transformers

Defaulting to user installation because normal site-packages is not writeable


In [4]:
clip_lables = df['clip_lables'].unique().tolist()
clip_lables

['Спортивное сооружение Динамо в Екатеринбург где Здание арафед с синим и белым внешним видом и синим забором',
 'Спортивное сооружение Динамо в Екатеринбург где Вид на воздушный вид на город с футбольным полем и большим апельсиновым куполом',
 'Спортивное сооружение Динамо в Екатеринбург где вид на здание с часовой башней на вершине',
 'Спортивное сооружение Динамо в Екатеринбург где Здание арафед с синим и белым знаком на нем',
 'Спортивное сооружение Динамо в Екатеринбург где Арафед здание с синим и белым знаком на его стороне',
 'Спортивное сооружение Динамо в Екатеринбург где вид на город с большим зданием и большим куполом',
 'Спортивное сооружение Динамо в Екатеринбург где Взгляд на город с футбольным полем и стадионом',
 'Спортивное сооружение Динамо в Екатеринбург где Взгляд на футбольное поле и реку в городе',
 'Спортивное сооружение Динамо в Екатеринбург где Вид на здание с башней посреди снежной зоны',
 'Спортивное сооружение Динамо в Екатеринбург где здание арафед с синим 

In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [6]:
def most_similar_sentence(input_string, sentences, df):
    """
    Определяет наиболее похожую строку из списка на входную строку по косинусному сходству.

    Args:
        input_string (string):  Строка, для которой ищем наиболее похожую.
        sentences (list): Список строк для сравнения.
    Returns:
        int: Индекс наиболее похожей строки в списке.
    """
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    all_sentences = [input_string] + sentences
    sentence_embeddings = model.encode(all_sentences)

    similarities = cosine_similarity([sentence_embeddings[0]], sentence_embeddings[1:])[0]

    most_similar_index = np.argmax(similarities)

    return (df[df['clip_lables'] == sentences[most_similar_index]]).index[0]

In [7]:
%%time

sentences = clip_lables
input_string = "Культурные учреждения Екатеринбурга, такие как Театр кукол, включая его здание с куполом и фасад, украшенный часами."

result = most_similar_sentence(input_string, sentences, df)
print(f"Наиболее похожая строка: {sentences[result]}")

Наиболее похожая строка: Театры и кинотеатры Театр кукол в Екатеринбург где Арафы и воздушные шары бросаются в воздух перед зданием
CPU times: total: 1min 43s
Wall time: 48.1 s


In [8]:
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

def prepare_embeddings(sentences, model_name='paraphrase-multilingual-MiniLM-L12-v2'):
    """
    Предварительно вычисляет эмбеддинги для заданного списка предложений и сохраняет их в файл.

    Args:
        sentences (list): Список строк для подготовки эмбеддингов.
        model_name (str): Название модели SentenceTransformer для эмбеддингов.

    Returns:
        None
    """
    model = SentenceTransformer(model_name)
    sentence_embeddings = model.encode(sentences)

    with open("precomputed_embeddings.pkl", "wb") as f:
        pickle.dump({"sentences": sentences, "embeddings": sentence_embeddings}, f)

def most_similar_sentence(input_string, df, embeddings_file="precomputed_embeddings.pkl"):
    """
    Определяет наиболее похожую строку из предварительно сохраненного списка на входную строку по косинусному сходству.

    Args:
        input_string (string): Строка, для которой ищем наиболее похожую.
        df (DataFrame): DataFrame с колонкой 'clip_lables'.
        embeddings_file (string): Путь к файлу с сохраненными эмбеддингами.

    Returns:
        int: Индекс наиболее похожей строки в DataFrame.
    """
    with open(embeddings_file, "rb") as f:
        data = pickle.load(f)

    sentences = data["sentences"]
    sentence_embeddings = data["embeddings"]

    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    input_embedding = model.encode([input_string])

    similarities = cosine_similarity(input_embedding, sentence_embeddings)[0]
    most_similar_index = np.argmax(similarities)

    return df[df['clip_lables'] == sentences[most_similar_index]].index[0]

In [9]:
%%time
input_string = "Культурные учреждения Екатеринбурга, такие как Театр кукол, включая его здание с куполом и фасад, украшенный часами."

prepare_embeddings(clip_lables) 

CPU times: total: 2min 1s
Wall time: 55.1 s


In [10]:
display(df.loc[result])

Name                                                 Театр кукол
WikiData                                                Q4227893
City                                                Екатеринбург
Lon                                                    60.610809
Lat                                                    56.839527
description                            кинотеатр в Екатеринбурге
image          /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
en_txt         rendering of a building with a clock on the to...
category                                     Театры и кинотеатры
ru_txt                      рендеринг здания с часами на вершине
clip_lables    Театры и кинотеатры Театр кукол в Екатеринбург...
Name: 93, dtype: object

In [11]:
%%time
input_string = "Культурные учреждения Екатеринбурга, такие как Театр кукол, включая его здание с куполом и фасад, украшенный часами."
result = most_similar_sentence(input_string, df)
display(df.loc[result])

Name                                                 Театр кукол
WikiData                                                Q4227893
City                                                Екатеринбург
Lon                                                    60.610809
Lat                                                    56.839527
description                            кинотеатр в Екатеринбурге
image          /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAUEBAUEAwUFBA...
en_txt         rendering of a building with a clock on the to...
category                                     Театры и кинотеатры
ru_txt                      рендеринг здания с часами на вершине
clip_lables    Театры и кинотеатры Театр кукол в Екатеринбург...
Name: 93, dtype: object

CPU times: total: 859 ms
Wall time: 3.16 s
